### Create Tools

In [9]:
from langchain_openai.chat_models.azure import AzureChatOpenAI
from typing import Annotated
from langchain_core.tools import tool
from langchain_experimental.utilities import PythonREPL
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai.chat_models.azure import AzureChatOpenAI
from typing import Literal
from typing_extensions import TypedDict
from langgraph.graph import MessagesState, END
from langgraph.types import Command
import os

openai_api_key = os.getenv('AZURE_OPENAI_API_KEY')
openai_api_base = os.getenv('AZURE_OPENAI_ENDPOINT')
openai_api_version = os.getenv('AZURE_OPENAI_API_VERSION')
embeddings_model = os.getenv('AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME')
gpt_model = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

index_path = "../api/index/faiss_index"

In [2]:
llm = AzureChatOpenAI(
                openai_api_version=openai_api_version,
                azure_deployment=gpt_model,
            )

embeddings = AzureOpenAIEmbeddings(azure_endpoint=openai_api_base,
                api_key=openai_api_key,
                azure_deployment=embeddings_model)

In [11]:
repl = PythonREPL()

@tool
def python_repl_tool(code: Annotated[str, "The python code to execute to generate charts"]):
    """Use this to execute python code and do math. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    try:
        result = repl.run([code])
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    result_str = f"Successfully executed: \n```python\n{code}\n```\nStdout: {result}"
    return result_str

In [5]:
@tool
def rag(state: AgentState):
        """Use this to execute RAG. If the question is related to gen ai in art or music, using this tool retrieve the results."""
        print("Calling RAG...")
        vector_store = FAISS.load_local(index_path, embeddings, allow_dangerous_deserialization=True)
        
        retriever = vector_store.as_retriever()
    
        template = """Answer questions based only on the following context: 
        
        ### CONTEXT ###
        {context}
        
        Question: {question}
        
        If the context doesn't provide enough information to answer the question, say that you don't know.
        Do not respond with information outside the previous context."""

        prompt = PromptTemplate.from_template(template)

        chain = (
            {"context": retriever, "question": RunnablePassthrough()}
            | prompt
            | llm
            | StrOutputParser()
        )

        result = chain.invoke(state)
        return result

### Create Agent Supervisor

### Construct Graph